In [1]:
# Import starting packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

# import datasets from the unzipped CSVs
actual_demand_file_path = '../data/totaldemand_nsw.csv'
temperature_file_path = '../data/temperature_nsw.csv'
actual_demand = pd.read_csv(actual_demand_file_path, parse_dates=['DATETIME'])
temperature = pd.read_csv(temperature_file_path, parse_dates=['DATETIME'])
forecast_demand_file_path = '../data/forecastdemand_nsw.csv'
forecast_demand = pd.read_csv(forecast_demand_file_path)

Exports_file_path = '../data/Exports - quarterly 2010-2023.csv'
GDP_file_path = '../data/GDP - quarterly 2010-2023.csv'
Interest_file_path = '../data/Interest rate - daily 2011-2023.csv'
SolarInstall_file_path = '../data/Solar installation number - monthly 2007-2023.csv'
SolarOutput_file_path = '../data/Solar outputs - monthly 2015-2023.csv'
Unemployment_file_path = '../data/Unemployment rate - monthly 2010-2023.csv'
Wholesale_file_path = '../data/Wholesale electricity price - half hourly 2014-2023.csv'
cpi_file_path = '../data/inflation-data.xls'
comm_file_path = '../data/Commodity prices index - monthly 2010-2023.csv'
rainfall_file_path = '../data/Rainfall - daily 2010-2023.csv'
weather_file_path = '../data/Weather - daily 2022-2023.csv'
public_holiday_path = '../data/nsw_public_holidays.csv'

cpi = pd.read_excel(cpi_file_path)
Exports = pd.read_csv(Exports_file_path)
GDP = pd.read_csv(GDP_file_path)
Interest = pd.read_csv(Interest_file_path)
SolarInstall = pd.read_csv(SolarInstall_file_path)
SolarOutput = pd.read_csv(SolarOutput_file_path)
Unemployment = pd.read_csv(Unemployment_file_path)
Wholesale = pd.read_csv(Wholesale_file_path)
Comm = pd.read_csv(comm_file_path)
Rainfall = pd.read_csv(rainfall_file_path)
Weather = pd.read_csv(weather_file_path)
public_holidays = pd.read_csv(public_holiday_path, parse_dates=['Date'])

In [2]:
lag_value = -6
# test_date = '2020-11-30'

In [3]:
## Uncomment this for 2022 and 2023 data combining
# path = '../data/nswdemand2022onwards'
# all_files = glob.glob(os.path.join(path, "*.csv"))

# total_demand_2022onwards = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
# total_demand_2022onwards.head()


# # Quick removal of certain columns and removal of incorrect -9999 values from the temperature data
# actual_demand.drop(columns='REGIONID', inplace=True)
# temperature.loc[temperature['TEMPERATURE'] <= -5, 'TEMPERATURE'] = np.nan
# temperature.drop(columns='LOCATION', inplace=True)

# # Combine temperature and demand
# df = pd.merge(actual_demand,temperature,left_on=['DATETIME'], right_on=['DATETIME'], how = 'left')

# # Interpolate missing temperature values
# df['temperature'] = df['TEMPERATURE'].interpolate(method='linear')

# # Remove extra temperature column and edit column names
# df.drop(columns='TEMPERATURE', inplace= True)
# df.rename(columns={'DATETIME': 'datetime', 'TOTALDEMAND':'demand'}, inplace=True)


# # Create date and time variables using the datetime column
# df['Is weekday'] = np.where(df['datetime'].dt.dayofweek <= 4, 1, 0)
# df['Is weekend'] = np.where(df['datetime'].dt.dayofweek > 4, 1, 0)

# for i in range(0,24):
#     df[f'hour_{i}'] = (df['datetime'].dt.hour == i).astype(int)

# for i in range(1,13):
#     df[f'month_{i}'] = (df['datetime'].dt.month == i).astype(int)


# ## CREATE Lag and summary features for use in models
# # create lag of demand and summaries of the demand lag to use as features in the model
# df['demand_5_min_lag'] = df['demand'].shift(1)
# df['demand_30_min_lag'] = df['demand'].shift(6)
# df['demand_1_hr_lag'] = df['demand'].shift(12)
# df['demand_24_hr_lag'] = df['demand'].shift(12*24)
# df['demand_30_min_mean'] = df['demand'].rolling(window = 6).mean()
# df['demand_1_hr_mean'] = df['demand'].rolling(window = 12).mean()
# df['demand_24_hr_mean'] = df['demand'].rolling(window = 12*24).mean()
# df['demand_30_min_std'] = df['demand'].rolling(window = 6).std()
# df['demand_1_hr_std'] = df['demand'].rolling(window = 12).std()
# df['demand_24_hr_std'] = df['demand'].rolling(window = 12*24).std()
# df['demand_30_min_max'] = df['demand'].rolling(window = 6).max()
# df['demand_1_hr_max'] = df['demand'].rolling(window = 12).max()
# df['demand_24_hr_max'] = df['demand'].rolling(window = 12*24).max()
# df['demand_30_min_min'] = df['demand'].rolling(window = 6).min()
# df['demand_1_hr_min'] = df['demand'].rolling(window = 12).min()
# df['demand_24_hr_min'] = df['demand'].rolling(window = 12*24).min()
# # create lag of temperature and summaries of the temperature lag to use as features in the model
# df['temperature_5_min_lag'] = df['temperature'].shift(1)
# df['temperature_30_min_lag'] = df['temperature'].shift(6)
# df['temperature_1_hr_lag'] = df['temperature'].shift(12)
# df['temperature_24_hr_lag'] = df['temperature'].shift(12*24)
# df['temperature_30_min_mean'] = df['temperature'].rolling(window = 6).mean()
# df['temperature_1_hr_mean'] = df['temperature'].rolling(window = 12).mean()
# df['temperature_24_hr_mean'] = df['temperature'].rolling(window = 12*24).mean()
# df['temperature_30_min_std'] = df['temperature'].rolling(window = 6).std()
# df['temperature_1_hr_std'] = df['temperature'].rolling(window = 12).std()
# df['temperature_24_hr_std'] = df['temperature'].rolling(window = 12*24).std()
# df['temperature_30_min_max'] = df['temperature'].rolling(window = 6).max()
# df['temperature_1_hr_max'] = df['temperature'].rolling(window = 12).max()
# df['temperature_24_hr_max'] = df['temperature'].rolling(window = 12*24).max()
# df['temperature_30_min_min'] = df['temperature'].rolling(window = 6).min()
# df['temperature_1_hr_min'] = df['temperature'].rolling(window = 12).min()
# df['temperature_24_hr_min'] = df['temperature'].rolling(window = 12*24).min()

# Add public holidays to the dataframe
# public_holiday_path = '../data/nsw_public_holidays.csv'
# public_holidays = pd.read_csv(public_holiday_path, parse_dates=['Date'])
# df['date'] = pd.to_datetime(df['datetime'].dt.date)
# df = pd.merge(df,public_holidays,left_on=['date'], right_on=['Date'], how = 'left')
# df['regular day'] = np.where(df['Holiday Name'].isnull(), 1, 0)
# df['public holiday'] = np.where(df['Holiday Name'].isnull(), 0, 1)
# df.drop(columns=['date', 'Date', 'Holiday Name'], inplace = True)

# Write to final_data.csv

In [5]:
# public_holidays = public_holidays.rename(columns=column_rename_mapping).dropna()
# public_holidays['DATETIME'] = pd.to_datetime(public_holidays['DATETIME'].dt.date)
# public_holidays = public_holidays.set_index('DATETIME')
# df = pd.merge(df,public_holidays, how = 'left')
# df['regular day'] = np.where(df['Holiday Name'].isnull(), 1, 0)
# df['public holiday'] = np.where(df['Holiday Name'].isnull(), 0, 1)
# df.drop(columns=['Date', 'Holiday Name'], inplace = True)

# column_rename_mapping = {
#     'Date': 'DATETIME',
# }
# public_holidays = public_holidays.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
# public_holidays.index = pd.to_datetime(public_holidays.index)
# date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
# min_date, max_date = df.index.min(), df.index.max()
# public_holidays = public_holidays[(public_holidays.index >= min_date) & (public_holidays.index <= max_date)]
# public_holidays = public_holidays.reindex(date_range).fillna(method='ffill')
# # df = pd.concat([df, cpi], axis=1).dropna()

In [7]:
temperature.loc[temperature['TEMPERATURE'] <= -5, 'TEMPERATURE'] = np.nan
df = pd.DataFrame({'Date Time': actual_demand['DATETIME'], 'Total Demand': actual_demand['TOTALDEMAND']})
df = pd.merge(df,temperature,left_on=['Date Time'], right_on=['DATETIME'], how = 'left')
df=df.drop(columns=['DATETIME', 'LOCATION'])
df['TEMPERATURE'] = df['TEMPERATURE'].interpolate(method='linear')
df['Date Time'] = pd.to_datetime(df['Date Time'])
forecast_demand['DATETIME'] = pd.to_datetime(forecast_demand['DATETIME'])
actual_demand['DATETIME'] = pd.to_datetime(actual_demand['DATETIME'])
forecast_demand['DATETIME'] = pd.to_datetime(forecast_demand['DATETIME'])
actual_demand['DATETIME'] = pd.to_datetime(actual_demand['DATETIME'])
fore = forecast_demand.set_index('DATETIME').resample('1H').mean()[['FORECASTDEMAND']]
actu = actual_demand.set_index('DATETIME').resample('1H').mean()[['TOTALDEMAND']]
merged_df = fore.join(actu).dropna()
merged_df['DIFF'] = merged_df.eval('TOTALDEMAND - FORECASTDEMAND')
merged_df['Demand_Previous'] = merged_df['TOTALDEMAND'].shift(periods=(-24))
merged_df['Demand_Lagged'] = merged_df['TOTALDEMAND'].shift(periods=(lag_value))
merged_df['Demand_Rolling'] = merged_df['Demand_Lagged'].rolling(window=720).mean()

temperature['DATETIME'] = pd.to_datetime(temperature['DATETIME'])
temp = temperature.set_index('DATETIME').resample('1H').mean()
temp['TEMPERATURE_lagged'] = temp['TEMPERATURE'].shift(periods=lag_value)
data = merged_df.merge(temp, left_index=True, right_index=True, how='inner')
data = data.drop(['LOCATION','TEMPERATURE', 'FORECASTDEMAND', 'DIFF'], axis=1)
df = data.dropna()


column_rename_mapping = {
    'G1 CONSUMER PRICE INFLATION': 'DATETIME',
    'Unnamed: 1': 'cpi'
}
cpi = cpi.rename(columns=column_rename_mapping)[10:].dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
cpi.index = pd.to_datetime(cpi.index, format='%b-%Y')
cpi = cpi[(cpi.index >= min_date) & (cpi.index <= max_date)]
cpi = cpi.reindex(date_range).fillna(method='ffill')
cpi['CPI_lagged'] = cpi['cpi'].shift(periods=lag_value)
cpi = cpi[['CPI_lagged']]
df = pd.concat([df, cpi], axis=1).dropna()

column_rename_mapping = {
    'Date': 'DATETIME',
    'Exports of goods and services': 'Exports'
}
Exports = Exports.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
Exports.index = pd.to_datetime(Exports.index, format='%b-%Y')
Exports = Exports[(Exports.index >= min_date) & (Exports.index <= max_date)]
Exports = Exports.reindex(date_range).fillna(method='ffill')
Exports['Exports_lagged'] = Exports['Exports'].shift(periods=lag_value)
Exports = Exports[['Exports_lagged']]
df = pd.concat([df, Exports], axis=1).dropna()

column_rename_mapping = {
    'Date': 'DATETIME',
    'Real GDP': 'GDP'
}
GDP = GDP.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
GDP.index = pd.to_datetime(GDP.index, format='%b-%Y')
GDP = GDP[(GDP.index >= min_date) & (GDP.index <= max_date)]
GDP = GDP.reindex(date_range).fillna(method='ffill')
GDP['GDP_lagged'] = GDP['GDP'].shift(periods=lag_value)
GDP = GDP[['GDP_lagged']]
df = pd.concat([df, GDP], axis=1).dropna()

column_rename_mapping = {
    'Date': 'DATETIME',
    'Cash Rate Target': 'Interest'
}
Interest = Interest.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
Interest.index = pd.to_datetime(Interest.index)
Interest = Interest[(Interest.index >= min_date) & (Interest.index <= max_date)]
Interest = Interest.reindex(date_range).fillna(method='ffill')
Interest['Interest_lagged'] = Interest['Interest'].shift(periods=lag_value)
Interest = Interest[['Interest_lagged']]
df = pd.concat([df, Interest], axis=1).dropna()

column_rename_mapping = {
    "'Month'": 'DATETIME',
    'Cumulative Residential': 'Residential',
    'Cumulative Commercial': 'Commercial'
}
SolarInstall = SolarInstall.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
SolarInstall.index = pd.to_datetime(SolarInstall.index)
SolarInstall = SolarInstall[(SolarInstall.index >= min_date) & (SolarInstall.index <= max_date)]
SolarInstall = SolarInstall.reindex(date_range).fillna(method='ffill')
SolarInstall['Residential_lagged'] = SolarInstall['Residential'].shift(periods=lag_value)
SolarInstall['Commercial_lagged'] = SolarInstall['Commercial'].shift(periods=lag_value)
SolarInstall = SolarInstall[['Residential_lagged','Commercial_lagged']]
df = pd.concat([df, SolarInstall], axis=1).dropna()

column_rename_mapping = {
    "Date": 'DATETIME',
    'Output NSW': 'SolarOutput'
}
SolarOutput = SolarOutput.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
SolarOutput.index = pd.to_datetime(SolarOutput.index)
SolarOutput = SolarOutput[(SolarOutput.index >= min_date) & (SolarOutput.index <= max_date)]
SolarOutput = SolarOutput.reindex(date_range).fillna(method='ffill')
SolarOutput['SolarOutput_lagged'] = SolarOutput['SolarOutput'].shift(periods=lag_value)
SolarOutput = SolarOutput[['SolarOutput_lagged']]
df = pd.concat([df, SolarOutput], axis=1).dropna()

column_rename_mapping = {
    "Date": 'DATETIME',
    'Unemployment rate': 'Unemployment'
}
Unemployment = Unemployment.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
Unemployment.index = pd.to_datetime(Unemployment.index, format='%b-%Y')
Unemployment = Unemployment[(Unemployment.index >= min_date) & (Unemployment.index <= max_date)]
Unemployment = Unemployment.reindex(date_range).fillna(method='ffill')
Unemployment['Unemployment_lagged'] = Unemployment['Unemployment'].shift(periods=lag_value)
Unemployment = Unemployment[['Unemployment_lagged']]
df = pd.concat([df, Unemployment], axis=1).dropna()

column_rename_mapping = {
    "SETTLEMENTDATE": 'DATETIME',
    'RRP': 'Price'
}
Wholesale = Wholesale.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
Wholesale.index = pd.to_datetime(Wholesale.index, format='%Y/%m/%d %H:%M:%S')
Wholesale = Wholesale[(Wholesale.index >= min_date) & (Wholesale.index <= max_date)]
Wholesale = Wholesale.reindex(date_range).fillna(method='ffill')
Wholesale['Price_lagged'] = Wholesale['Price'].shift(periods=lag_value)
Wholesale = Wholesale[['Price_lagged']]
df = pd.concat([df, Wholesale], axis=1).dropna()

column_rename_mapping = {
    "Date": 'DATETIME',
    'Commodity prices ': 'CommPrice'
}
Comm = Comm.rename(columns=column_rename_mapping).dropna().set_index('DATETIME')
min_date, max_date = data.index.min(), data.index.max()
date_range = pd.date_range(start=min_date, end=max_date,freq='1H')
Comm.index = pd.to_datetime(Comm.index, format='%b-%Y')
Comm = Comm[(Comm.index >= min_date) & (Comm.index <= max_date)]
Comm = Comm.reindex(date_range).fillna(method='ffill')
Comm['CommPrice_lagged'] = Comm['CommPrice'].shift(periods=lag_value)
Comm = Comm[['CommPrice_lagged']]
df = pd.concat([df, Comm], axis=1).dropna()
df['HourOfDay'] = df.index.hour
day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['DayOfWeek'] = df.index.dayofweek.map(day_mapping)
df['DayOfYear'] = df.index.dayofyear
df['IsWeekend'] = np.where((df['DayOfWeek'] == 5) | (df['DayOfWeek'] == 6), 1, 0)
# df = pd.get_dummies(df, columns=['HourOfDay'], prefix='Hour')
# df = pd.get_dummies(df, columns=['DayOfWeek'], prefix='Day')
# df = pd.get_dummies(df, columns=['DayOfYear'], prefix='Year')
# df = pd.get_dummies(df, columns=['IsWeekend'], prefix='Weekend')

In [9]:
df.to_csv('../data/final_data.csv',index=False, mode = 'w')

df.head()

,TOTALDEMAND,Demand_Previous,Demand_Lagged,Demand_Rolling,TEMPERATURE_lagged,CPI_lagged,Exports_lagged,GDP_lagged,Interest_lagged,Residential_lagged,Commercial_lagged,SolarOutput_lagged,Unemployment_lagged,Price_lagged,CommPrice_lagged,HourOfDay,DayOfWeek,DayOfYear,IsWeekend
2015-03-31 18:00:00,8467.307500,8738.001667,6833.315833,7767.094856,14.666667,106.8,1.081540e+11,4.632940e+11,2.25,765282.0,291300.0,11308.0,6.1,33.59,45.6,18,Tuesday,90,0
2015-03-31 19:00:00,8160.216667,8448.161667,6331.099167,7767.564594,15.825000,106.8,1.081540e+11,4.632940e+11,2.25,765282.0,291300.0,11308.0,6.1,27.02,45.6,19,Tuesday,90,0
2015-03-31 20:00:00,7678.155000,7952.970000,5947.957500,7767.780329,16.283333,106.8,1.081540e+11,4.632940e+11,2.25,765282.0,291300.0,11308.0,6.1,27.70,45.6,20,Tuesday,90,0
2015-03-31 21:00:00,7462.178333,7671.011667,5931.215000,7767.914020,15.700000,106.8,1.081540e+11,4.632940e+11,2.25,765282.0,291300.0,11308.0,6.1,20.96,45.6,21,Tuesday,90,0
2015-03-31 22:00:00,7299.590833,7463.340000,6303.179167,7768.012985,14.900000,106.8,1.081540e+11,4.632940e+11,2.25,765282.0,291300.0,11308.0,6.1,24.63,45.6,22,Tuesday,90,0
